# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [1]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [2]:
from langchain.chains import RetrievalQA
from langchain_chroma import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

llm = OpenAI(temperature=0)

In [3]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "state_of_the_union.txt")

In [4]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")

In [5]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [6]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [8]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

Created a chunk of size 2122, which is longer than the specified 1000
Created a chunk of size 3187, which is longer than the specified 1000
Created a chunk of size 1017, which is longer than the specified 1000
Created a chunk of size 1049, which is longer than the specified 1000
Created a chunk of size 1256, which is longer than the specified 1000
Created a chunk of size 2321, which is longer than the specified 1000


## Create the Agent

In [9]:
# Import things that are needed generically
from langchain.agents import Tool

In [10]:
tools = [
    Tool(
        name="state_of_union_qa_system",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
    ),
    Tool(
        name="ruff_qa_system",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [11]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent("openai:gpt-4.1-mini", tools)

In [12]:
input_message = {
    "role": "user",
    "content": "What did biden say about ketanji brown jackson in the state of the union address?",
}

for step in agent.stream(
    {"messages": [input_message]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What did biden say about ketanji brown jackson in the state of the union address?
================================== Ai Message ==================================
Tool Calls:
  state_of_union_qa_system (call_26QlRdsptjEJJZjFsAUjEbaH)
 Call ID: call_26QlRdsptjEJJZjFsAUjEbaH
  Args:
    __arg1: What did Biden say about Ketanji Brown Jackson in the state of the union address?
================================= Tool Message =================================
Name: state_of_union_qa_system

 Biden said that he nominated Ketanji Brown Jackson for the United States Supreme Court and praised her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.
================================== Ai Message ==================================

In the State of the Union address, Biden said that he nominated Ketanji Brown Jackson for the United States Supreme Court and praised her as one of

In [13]:
input_message = {
    "role": "user",
    "content": "Why use ruff over flake8?",
}

for step in agent.stream(
    {"messages": [input_message]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Why use ruff over flake8?
================================== Ai Message ==================================
Tool Calls:
  ruff_qa_system (call_KqDoWeO9bo9OAXdxOsCb6msC)
 Call ID: call_KqDoWeO9bo9OAXdxOsCb6msC
  Args:
    __arg1: Why use ruff over flake8?
================================= Tool Message =================================
Name: ruff_qa_system


There are a few reasons why someone might choose to use Ruff over Flake8:

1. Larger rule set: Ruff implements over 800 rules, while Flake8 only implements around 200. This means that Ruff can catch more potential issues in your code.

2. Better compatibility with other tools: Ruff is designed to work well with other tools like Black, isort, and type checkers like Mypy. This means that you can use Ruff alongside these tools to get more comprehensive feedback on your code.

3. Automatic fixing of lint violations: Unlike Flake8, Ruff is capable of automati

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [14]:
tools = [
    Tool(
        name="state_of_union_qa_system",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="ruff_qa_system",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [15]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent("openai:gpt-4.1-mini", tools)

In [16]:
input_message = {
    "role": "user",
    "content": "What did biden say about ketanji brown jackson in the state of the union address?",
}

for step in agent.stream(
    {"messages": [input_message]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What did biden say about ketanji brown jackson in the state of the union address?
================================== Ai Message ==================================
Tool Calls:
  state_of_union_qa_system (call_yjxh11OnZiauoyTAn9npWdxj)
 Call ID: call_yjxh11OnZiauoyTAn9npWdxj
  Args:
    __arg1: What did Biden say about Ketanji Brown Jackson in the state of the union address?
================================= Tool Message =================================
Name: state_of_union_qa_system

 Biden said that he nominated Ketanji Brown Jackson for the United States Supreme Court and praised her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.


In [17]:
input_message = {
    "role": "user",
    "content": "Why use ruff over flake8?",
}

for step in agent.stream(
    {"messages": [input_message]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Why use ruff over flake8?
================================== Ai Message ==================================
Tool Calls:
  ruff_qa_system (call_GiWWfwF6wbbRFQrHlHbhRtGW)
 Call ID: call_GiWWfwF6wbbRFQrHlHbhRtGW
  Args:
    __arg1: What are the advantages of using ruff over flake8 for Python linting?
================================= Tool Message =================================
Name: ruff_qa_system

 Ruff has a larger rule set, supports automatic fixing of lint violations, and does not require the installation of additional plugins. It also has better compatibility with Black and can be used alongside a type checker for more comprehensive code analysis.


## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [18]:
tools = [
    Tool(
        name="state_of_union_qa_system",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="ruff_qa_system",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [19]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent("openai:gpt-4.1-mini", tools)

In [20]:
input_message = {
    "role": "user",
    "content": "What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?",
}

for step in agent.stream(
    {"messages": [input_message]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?
================================== Ai Message ==================================
Tool Calls:
  ruff_qa_system (call_VOnxiOEehauQyVOTjDJkR5L2)
 Call ID: call_VOnxiOEehauQyVOTjDJkR5L2
  Args:
    __arg1: What tool does ruff use to run over Jupyter Notebooks?
  state_of_union_qa_system (call_AbSsXAxwe4JtCRhga926SxOZ)
 Call ID: call_AbSsXAxwe4JtCRhga926SxOZ
  Args:
    __arg1: Did the president mention the tool that ruff uses to run over Jupyter Notebooks in the state of the union?
================================= Tool Message =================================
Name: state_of_union_qa_system

 No, the president did not mention the tool that ruff uses to run over Jupyter Notebooks in the state of the union.
================================== Ai Message ==================================

Ruff d